In [2]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage, AnyMessage
from IPython.display import Image, display
from langgraph.graph import MessagesState,StateGraph, START, END
from typing import TypedDict, Annotated, List
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langchain_ollama import ChatOllama

load_dotenv()

# Load the environment variables
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = os.getenv("MODEL")

ollamaModel = ChatOllama(
    model="llama3.1",
    temperature=0,
)

llm = ChatOpenAI(model=MODEL, api_key=OPENAI_API_KEY, base_url=OPENAI_API_BASE, temperature=0)

In [3]:
system_message = SystemMessage(content="You are a helpfule assistant tasked with performing arithmatic on a set of inputs.")


In [5]:
def multiply(a: float, b: float) -> float:
    """Multiply a and b
    
    Args:
        a: first int
        b: second int
    """

    print(f"Custom Tool Multiplying {a} by {b}")
    return a*b

def add(a: float, b: float) -> float:
    """Add a and b
    
    Args:
        a: first int
        b: second int
    """
    print(f"Custom Tool Adding {a} and {b}")
    return a+b

def subtract(a: float, b: float) -> float:
    """Subtract b from a
    
    Args:
        a: first int
        b: second int
    """
    print(f"Custom Tool Subtracting {b} from {a}")
    return a-b

def divide(a: float, b: float) -> float:
    """Divide a by b
    
    Args:
        a: first int
        b: second int
    """
    print(f"Custom Tool Dividing {a} by {b}")
    return a/b

In [12]:
#tools = [add, subtract, multiply, divide]

#llm_with_tools = llm.bind_tools(tools)

In [6]:
tools = [add, subtract, multiply, divide]

llm_with_tools = ollamaModel.bind_tools(tools)

In [7]:
class Assistant:
    def __init__(self, llm_with_tools):
        """
        Initialize the Assistant with llm. 
        """

        self.llm_with_tools = llm_with_tools

    def __call__(self, state: MessagesState, config):
        """
        Call method to invoke
        """
        # Get the last message
        messsages = state['messages']

        result = self.llm_with_tools.invoke([system_message]+messsages)
        return {"messages":result}
        
        
    

In [13]:
# Graph
builder = StateGraph(MessagesState)

#Define the nodes
builder.add_node("assistant", Assistant(ollamaModel.bind_tools(tools)))
builder.add_node("tools",ToolNode(tools))

# Define the edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)

builder.add_edge("tools","assistant")
react_graph = builder.compile()

In [ ]:
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))


In [ ]:
messages = [HumanMessage(content="Add 3 and 4, then multiple by 2, and finally divide by 5")]
messages = react_graph.invoke({"messages":messages})


In [ ]:
for message in messages['messages']:
    message.pretty_print()